In [17]:
import yfinance as yf
import datetime
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

file_path = 'EA_stock_data.csv'
ea_df = pd.read_csv(file_path)

<h1>SVM WITHOUT TA,FEATURE REDUCTION AND ADABOOST</h1>

In [4]:
file_path = 'EA_stock_data.csv'
ea_df = pd.read_csv(file_path)
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

ea_df.dropna(inplace=True)
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.1).mean()
features = ['Open', 'High', 'Low', 'Close', 'Volume','Smoothed_Close']
X = ea_df[features]
ea_df.dropna(inplace=True)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = ea_df['Target'].values

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)
svm = SVC(C=9.699920698680053,class_weight='balanced',degree=5,coef0=6.080765165859437 ,kernel='linear', gamma=0.02997773203425469, probability=True, random_state=42)
svm.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

Accuracy: 0.8531746031746031
              precision    recall  f1-score   support

           0       0.82      0.96      0.89       152
           1       0.92      0.69      0.79       100

    accuracy                           0.85       252
   macro avg       0.87      0.83      0.84       252
weighted avg       0.86      0.85      0.85       252



<h1>CV FOR SVM WITHOUT TA AND ADABOOST</h1>

In [5]:
from sklearn.model_selection import TimeSeriesSplit


tscv = TimeSeriesSplit(n_splits=10)

# Initialize the SVM model
svm = SVC(C=9.699920698680053, kernel='linear', degree=5, coef0=6.080765165859437, gamma=0.02997773203425469, class_weight='balanced', probability=True, random_state=42)

# List to store scores for each fold
accuracy_scores = []

# Perform the time series cross-validation
for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the model
    svm.fit(X_train, y_train)
    
    # Make predictions
    y_pred = svm.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    
    # Print the classification report for each fold
    print(classification_report(y_test, y_pred, zero_division=0))

# Average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(f'Average Accuracy across folds: {average_accuracy}')

              precision    recall  f1-score   support

           0       0.17      1.00      0.29        16
           1       1.00      0.21      0.35        98

    accuracy                           0.32       114
   macro avg       0.59      0.61      0.32       114
weighted avg       0.88      0.32      0.34       114

              precision    recall  f1-score   support

           0       0.15      1.00      0.27        16
           1       1.00      0.10      0.19        98

    accuracy                           0.23       114
   macro avg       0.58      0.55      0.23       114
weighted avg       0.88      0.23      0.20       114

              precision    recall  f1-score   support

           0       0.53      0.83      0.65        36
           1       0.89      0.65      0.76        78

    accuracy                           0.71       114
   macro avg       0.71      0.74      0.70       114
weighted avg       0.78      0.71      0.72       114

              preci

<H1>SVM WITH TA AND FEATURE REDUCTION</H1>

In [11]:
import ta  # Import the technical analysis library
file_path = 'EA_stock_data.csv'
ea_df = pd.read_csv(file_path)
# Assuming ea_df is your DataFrame loaded with stock data including 'Open', 'High', 'Low', 'Close', 'Volume'

# Calculate Moving Average
ea_df['SMA_10'] = ea_df['Close'].rolling(window=20).mean()
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=14).adx()  
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.2).mean()


# Define the target variable: 1 if the price goes up next week, 0 otherwise
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')
ea_df.dropna(inplace=True)
# Update features list to include the new technical indicators
features = ['Close', 'SMA_10', 'MACD', 'ADX','Smoothed_Close']
X = ea_df[features]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = ea_df['Target'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# Initialize and train the SVM model
svm = SVC(C=0.7400230211761833, kernel='rbf',degree=4,coef0=0.09111040120352687, gamma=0.2002723939261901, class_weight='balanced', probability=True, random_state=42)
svm.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

Accuracy: 0.8947368421052632
              precision    recall  f1-score   support

           0       0.87      0.98      0.92       152
           1       0.96      0.76      0.85        95

    accuracy                           0.89       247
   macro avg       0.91      0.87      0.88       247
weighted avg       0.90      0.89      0.89       247



In [8]:
from sklearn.model_selection import TimeSeriesSplit


tscv = TimeSeriesSplit(n_splits=10)

# Initialize the SVM model
svm = SVC(C=0.7400230211761833, kernel='rbf', degree=4, coef0=0.09111040120352687, gamma=0.2002723939261901, class_weight='balanced', probability=True, random_state=42)

# List to store scores for each fold
accuracy_scores = []

# Perform the time series cross-validation
for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the model
    svm.fit(X_train, y_train)
    
    # Make predictions
    y_pred = svm.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    
    # Print the classification report for each fold
    print(classification_report(y_test, y_pred, zero_division=0))

# Average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(f'Average Accuracy across folds: {average_accuracy}')

              precision    recall  f1-score   support

           0       0.21      1.00      0.34        16
           1       1.00      0.35      0.52        96

    accuracy                           0.45       112
   macro avg       0.60      0.68      0.43       112
weighted avg       0.89      0.45      0.50       112

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        32
           1       0.67      0.81      0.73        80

    accuracy                           0.58       112
   macro avg       0.34      0.41      0.37       112
weighted avg       0.48      0.58      0.52       112

              precision    recall  f1-score   support

           0       0.49      1.00      0.66        22
           1       1.00      0.74      0.85        90

    accuracy                           0.79       112
   macro avg       0.74      0.87      0.76       112
weighted avg       0.90      0.79      0.81       112

              preci

<h1>svm with ta and ada boost</h1>

In [13]:
import ta  # Import the technical analysis library
file_path = 'EA_stock_data.csv'
ea_df = pd.read_csv(file_path)
# Assuming ea_df is your DataFrame loaded with stock data including 'Open', 'High', 'Low', 'Close', 'Volume'

ea_df['SMA_10'] = ea_df['Close'].rolling(window=10).mean()
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=14).adx()
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.2).mean()
# Drop any rows with NaN values that were introduced by the indicator calculations

# Define the target variable: 1 if the price goes up next week, 0 otherwise
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')
ea_df.dropna(inplace=True)

# Update features list to include the new technical indicators
features = [ 'High', 'Low', 'Close', 'Volume', 'SMA_10', 'MACD', 'ADX','Smoothed_Close']
X = ea_df[features]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = ea_df['Target'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# Initialize and train the SVM model
svm = SVC(C=0.7400230211761833, kernel='rbf',degree=4,coef0=0.09111040120352687, gamma=0.2002723939261901, class_weight='balanced', probability=True, random_state=42)
svm.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

Accuracy: 0.8866396761133604
              precision    recall  f1-score   support

           0       0.85      0.99      0.92       152
           1       0.99      0.72      0.83        95

    accuracy                           0.89       247
   macro avg       0.92      0.85      0.87       247
weighted avg       0.90      0.89      0.88       247



In [14]:
from sklearn.ensemble import AdaBoostClassifier
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# Initialize the SVM model with probability=True for AdaBoost compatibility
svm_base = SVC(C=8, kernel='rbf', gamma=0.01, class_weight=None, probability=True, random_state=42)

# Initialize and train the AdaBoost model with SVM as the base estimator
ada_boost_model = AdaBoostClassifier(base_estimator=svm_base, n_estimators=50, random_state=42)
ada_boost_model.fit(X_train, y_train)

# Make predictions and evaluate the AdaBoost model
y_pred = ada_boost_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

Accuracy: 0.8906882591093117
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       152
           1       1.00      0.72      0.83        95

    accuracy                           0.89       247
   macro avg       0.92      0.86      0.88       247
weighted avg       0.91      0.89      0.89       247



<h1>CV FOR SVM WITH TA AND ADABOOST</h1>


In [16]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score

# Load your data and preprocess as you have specified
# Code for data loading and preprocessing is assumed to be done here

# Define the time series cross-validator
tscv = TimeSeriesSplit(n_splits=10)

# Initialize the SVM model with probability=True for AdaBoost compatibility
svm_base = SVC(C=8, kernel='rbf', gamma=0.01, class_weight=None, probability=True, random_state=42)

# Initialize the AdaBoost model with SVM as the base estimator
ada_boost_model = AdaBoostClassifier(base_estimator=svm_base, n_estimators=50, random_state=42)

# List to store scores for each fold
accuracy_scores = []

# Perform the time series cross-validation
for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the AdaBoost model
    ada_boost_model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = ada_boost_model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    
    # Print the classification report for each fold
    print(classification_report(y_test, y_pred, zero_division=0))

# Average accuracy across all folds
average_accuracy = np.mean(accuracy_scores)
print(f'Average Accuracy across folds: {average_accuracy}')

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.86      1.00      0.92        96

    accuracy                           0.86       112
   macro avg       0.43      0.50      0.46       112
weighted avg       0.73      0.86      0.79       112

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        32
           1       0.71      1.00      0.83        80

    accuracy                           0.71       112
   macro avg       0.36      0.50      0.42       112
weighted avg       0.51      0.71      0.60       112

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       0.80      1.00      0.89        90

    accuracy                           0.80       112
   macro avg       0.40      0.50      0.45       112
weighted avg       0.65      0.80      0.72       112

              preci